In [1]:
import os
import re
from bs4 import BeautifulSoup
import requests
import json

In [2]:
articles_dir = 'Articles'
annotation_dir = 'Annotations'

if not os.path.exists(articles_dir):
    os.makedirs(path)

if not os.path.exists(annotation_dir):
    os.makedirs(path)

## Download all the articles

In [3]:
overview_pages = ['https://www.nytimes.com/interactive/2014/12/29/us/year-in-interactive-storytelling.html',
 'https://www.nytimes.com/interactive/2015/us/year-in-interactive-storytelling.html',
 'https://www.nytimes.com/interactive/2016/12/28/us/year-in-interactive-graphics.html',
 'https://www.nytimes.com/interactive/2017/12/21/us/2017-year-in-graphics.html',
 'https://www.nytimes.com/interactive/2018/us/2018-year-in-graphics.html',
 'https://www.nytimes.com/interactive/2019/12/30/us/2019-year-in-graphics.html',
 'https://www.nytimes.com/interactive/2020/12/30/us/2020-year-in-graphics.html',
 'https://www.nytimes.com/interactive/2021/12/29/us/2021-year-in-graphics.html',
 'https://www.nytimes.com/interactive/2022/12/28/us/2022-year-in-graphics.html']

In [4]:

all_links = []
for page in overview_pages:
    response = requests.get(page)
    if response:
        html_page = response.text
        soup = BeautifulSoup( html_page ,"lxml")
        links = soup.find_all('a')
        for l in links: 
            url = l.get('href')
            if re.search( r'20\d{2}' , url ) and not(re.search('year-in-graphics',url)):
                all_links.append(url)

all_links = list(set(all_links))

In [5]:
for article in all_links:
    print(article)
    name =  os.path.basename(article)
    if re.search( r'\?' , name ):
        ## Url with parameters
        html_index = article.index('.html')
        name = article[:html_index]
        name += '.html' 
        name =  os.path.basename(name)
    elif not(re.search(r'html?$',name)):
        ## Link to a folder with index page
        name = re.sub( r'[\/]$' , '' , article )
        name =  name[ name.rindex('/')+1 : ]
        name += '.html'
        print(article)
        print(name)

    path = os.path.join( articles_dir , name )
    with open( path , 'w' , encoding = 'utf-8') as out:
        response = requests.get(article)
        if response:
            out.write(response.text)
    

https://www.nytimes.com/interactive/2018/09/13/us/hurricane-florence-impact-damage-map.html
http://www.nytimes.com/interactive/2015/09/03/magazine/migrants.html
http://www.nytimes.com/interactive/2016/04/05/us/elections/state-voter-histograms.html
https://www.nytimes.com/interactive/2017/06/08/arts/frank-lloyd-wright-at-150-moma.html
https://www.nytimes.com/interactive/2022/12/24/upshot/tom-brady-of-other-jobs.html
https://www.nytimes.com/interactive/2017/05/23/world/europe/what-happened-at-the-manchester-arena-explosion.html
https://www.nytimes.com/interactive/2019/09/14/world/europe/notre-dame-fire-lead.html
https://www.nytimes.com/interactive/2021/us/cdc-mask-guidance-states.html
https://www.nytimes.com/interactive/2022/06/12/us/guantanamo-bay-pentagon-photos.html
https://www.nytimes.com/interactive/2018/08/02/us/chicago-river-waterfront.html
https://www.nytimes.com/interactive/2018/05/24/us/disasters-hurricanes-wildfires-storms.html
https://www.nytimes.com/interactive/2018/08/30/cl

http://www.nytimes.com/interactive/2015/11/15/world/europe/manhunt-for-paris-attackers.html
https://www.nytimes.com/interactive/2019/01/30/science/polar-vortex-extreme-cold.html
https://www.nytimes.com/interactive/2017/06/14/world/europe/migrant-rescue-efforts-deadly.html
https://www.nytimes.com/interactive/2021/12/13/opinion/climate-change-effects-countries.html
https://www.nytimes.com/interactive/2019/05/11/us/school-shootings-united-states.html
https://www.nytimes.com/interactive/2017/06/15/world/europe/climate-change-rotterdam.html
https://www.nytimes.com/interactive/2020/10/08/us/white-house-coronavirus-cdc.html
https://www.nytimes.com/interactive/2017/11/07/dining/400-degree-thanksgiving.html
https://www.nytimes.com/interactive/2020/12/03/opinion/covid-19-vaccine-timeline.html?action=click&module=Opinion&pgtype=Homepage
https://www.nytimes.com/interactive/2017/06/22/climate/95-degree-day-maps.html
https://www.nytimes.com/interactive/2021/04/30/world/asia/india-coronavirus-family.

http://www.nytimes.com/interactive/2016/06/05/magazine/new-york-life.html
https://www.nytimes.com/interactive/2021/04/05/us/coronavirus-pandemic.html
https://www.nytimes.com/interactive/2021/06/12/world/americas/mexico-city-train-crash.html
https://www.nytimes.com/interactive/2018/11/18/us/california-camp-fire-paradise.html
http://www.nytimes.com/interactive/2014/06/27/us/detroit-foreclosure-photo-mosaic.html
https://www.nytimes.com/interactive/2018/02/21/sports/olympics/downhill-lindsey-vonn-sofia-goggia.html
http://www.nytimes.com/interactive/2014/08/13/upshot/where-people-in-each-state-were-born.html
https://www.nytimes.com/interactive/2021/06/22/technology/xinjiang-uyghurs-china-propaganda.html
https://www.nytimes.com/interactive/2021/01/19/opinion/trump-legacy-headlines.html
https://www.nytimes.com/interactive/2021/08/12/us/2020-census-race-ethnicity.html
https://www.nytimes.com/interactive/2021/05/15/world/covid-inequality-vaccines.html
https://www.nytimes.com/interactive/2020/11

https://www.nytimes.com/interactive/2019/10/14/us/politics/democrats-political-facebook-ads.html
https://www.nytimes.com/interactive/2021/01/15/world/europe/england-covid-hospitals.html
https://www.nytimes.com/interactive/2017/01/21/world/womens-march-pictures.html
http://www.nytimes.com/interactive/2016/12/09/science/mapping-three-decades-of-global-water-change.html
https://www.nytimes.com/interactive/2022/world/asia/taiwan-china-maps.html
https://www.nytimes.com/interactive/2020/07/05/us/coronavirus-latinos-african-americans-cdc-data.html
https://www.nytimes.com/interactive/2022/07/26/us/american-sign-language-changes.html
https://www.nytimes.com/interactive/2018/02/15/us/school-shootings-sandy-hook-parkland.html
http://www.nytimes.com/interactive/2016/05/22/world/europe/europe-right-wing-austria-hungary.html
https://www.nytimes.com/interactive/2017/02/27/upshot/whats-normal-whats-important-a-ranking-of-20-events-in-the-trump-administration.html
http://www.nytimes.com/interactive/201

http://www.nytimes.com/interactive/2015/03/24/world/europe/germanwings-plane-crash-map.html
https://www.nytimes.com/interactive/2018/10/25/multimedia/bugs-halloween-kids-ar-ul.html
https://www.nytimes.com/interactive/2018/05/06/us/puerto-rico-power-grid-hurricanes.html
https://www.nytimes.com/interactive/2017/12/18/sports/everest-deaths.html
https://www.nytimes.com/interactive/2018/05/16/upshot/audio-clip-yanny-laurel-debate.html
https://www.nytimes.com/interactive/2019/10/23/nyregion/basements-queens-immigrants.html
https://www.nytimes.com/interactive/2019/04/17/climate/switzerland-glaciers-climate-change.html
http://www.nytimes.com/newsgraphics/2014/sochi-olympics/halfpipe.html
https://www.nytimes.com/interactive/2017/01/18/upshot/some-colleges-have-more-students-from-the-top-1-percent-than-the-bottom-60.html
https://www.nytimes.com/interactive/2021/11/30/world/europe/europe-covid-surge-omicron.html
https://www.nytimes.com/interactive/2019/04/19/us/politics/redacted-mueller-report.ht

...

https://www.nytimes.com/interactive/2017/08/24/us/affirmative-action.html
https://www.nytimes.com/interactive/2021/04/18/climate/climate-change-future-kids.html
https://www.nytimes.com/interactive/2021/06/29/upshot/portland-seattle-vancouver-weather.html
https://www.nytimes.com/interactive/2018/11/06/us/elections/results-california-elections.html
https://www.nytimes.com/interactive/2022/sports/baseball/umpire-pitch-ball-strike-game.html
https://www.nytimes.com/interactive/2022/06/24/us/politics/supreme-court-dobbs-jackson-analysis-roe-wade.html
https://www.nytimes.com/2020/05/31/us/george-floyd-investigation.html
https://www.nytimes.com/interactive/2018/05/12/us/kilauea-volcano-lava-leilani-estates-hawaii.html
https://www.nytimes.com/interactive/2019/03/19/us/midwest-flooding.html
http://www.nytimes.com/interactive/2016/08/10/sports/olympics/gymnastics-parents.html
http://www.nytimes.com/interactive/2016/07/22/us/politics/trump-immigration-ban-how-could-it-work.html
https://www.nytimes

https://www.nytimes.com/interactive/2019/06/07/sports/soccer/world-cup-survey.html
https://www.nytimes.com/interactive/2022/11/29/sports/world-cup/iran-usa-world-cup-goal-pulisic.html
http://www.nytimes.com/interactive/2016/02/09/us/politics/campaign-ad-tracking.html
https://www.nytimes.com/interactive/2018/06/27/world/americas/venezuela-money-how-to-survive.html
http://www.nytimes.com/interactive/2016/11/23/us/dakota-access-pipeline-protest-map.html
https://www.nytimes.com/interactive/2021/upshot/2020-election-map.html
https://www.nytimes.com/interactive/2017/11/05/world/middleeast/kurd-independence-iraq-turkey-iran-syria-maps-backfired.html
https://www.nytimes.com/interactive/2018/06/07/magazine/new-york-love-city.html
https://www.nytimes.com/interactive/2021/01/27/nyregion/brooklyn-streets-numbers-renaming.html
http://www.nytimes.com/interactive/2014/10/27/us/is-the-affordable-care-act-working.html
https://www.nytimes.com/interactive/2019/06/20/world/asia/hong-kong-protest-size.html

https://www.nytimes.com/interactive/2018/06/25/world/middleeast/syria-chemical-attack-douma.html
https://www.nytimes.com/interactive/2022/09/08/opinion/urban-highways-segregation.html
https://www.nytimes.com/interactive/2021/08/18/opinion/inflation-economy-transitory.html
https://www.nytimes.com/interactive/2017/12/05/us/politics/tax-bill-salt.html
https://www.nytimes.com/interactive/2021/09/05/us/caldor-fire-lake-tahoe.html
https://www.nytimes.com/interactive/2020/07/26/nyregion/nyc-covid-19-mta-transit-workers.html
https://www.nytimes.com/interactive/2020/08/28/us/politics/rnc-speakers-time.html
http://www.nytimes.com/interactive/2014/12/11/world/africa/boko-haram-nigeria-maps.html
https://www.nytimes.com/interactive/2020/07/25/world/americas/coronavirus-brazil-amazon.html
http://www.nytimes.com/interactive/2016/08/24/world/europe/italy-earthquake-before-after-amatrice-accumoli.html
https://www.nytimes.com/interactive/2018/06/29/sports/world-cup/kroos-ronaldo-world-cup-goals.html
htt

https://www.nytimes.com/interactive/2017/04/12/world/asia/north-korea-nuclear-test.html
https://www.nytimes.com/interactive/2019/12/02/climate/air-pollution-compare-ar-ul.html
https://www.nytimes.com/interactive/2020/us/states-reopen-map-coronavirus.html
https://www.nytimes.com/interactive/2020/05/05/us/coronavirus-death-toll-us.html
https://www.nytimes.com/2020/12/03/arts/music/prince-sign-o-the-times.html
https://www.nytimes.com/interactive/2017/10/14/us/california-fire-building-damage.html
http://www.nytimes.com/interactive/2015/05/21/world/middleeast/how-isis-expands.html
https://www.nytimes.com/interactive/2021/health/pfizer-coronavirus-vaccine.html
https://www.nytimes.com/interactive/2021/05/24/us/tulsa-race-massacre.html
https://www.nytimes.com/interactive/2018/03/14/climate/easter-island-erosion.html
https://www.nytimes.com/interactive/2015/12/27/sunday-review/2015-year-in-pictures.html
http://www.nytimes.com/newsgraphics/2014/sochi-olympics/short-track.html
http://www.nytimes.

https://www.nytimes.com/interactive/2018/06/27/us/politics/kennedy-retirement-supreme-court-median.html
http://www.nytimes.com/interactive/2014/12/30/business/a-record-year-for-auto-recalls.html
https://www.nytimes.com/interactive/2019/04/17/world/europe/notre-dame-cathedral-fire-spread.html
http://www.nytimes.com/interactive/2014/06/22/nyregion/rookie-new-york-firefighter-faces-first-test.html
https://www.nytimes.com/interactive/2019/09/06/us/hurricane-dorian-path-map-track.html
https://www.nytimes.com/interactive/2021/12/18/us/vaccine-mandate-states.html
https://www.nytimes.com/interactive/2022/04/14/upshot/tonga-pressure-wave.html
http://www.nytimes.com/interactive/2016/07/25/us/politics/political-firsts.html
https://www.nytimes.com/interactive/2019/12/17/us/oakland-california-homeless-camp.html
http://www.nytimes.com/interactive/2015/07/15/world/middleeast/iran-deal-qa.html
http://www.nytimes.com/interactive/2016/03/15/arts/music/george-martin-fifth-beatle.html
https://www.nytimes.

## Identification of modalities

In [6]:
def get_attribute_value(attribute):
    attributes = element.get(attribute)
    if type(attributes) == list:
        return ' '.join(attributes)
    else:
        return attributes
    
def analyse_transitions(components):
    transitions = []
    current = ''
    for c in components:
        if c != current:
            transitions.append(c)
            current = c
    return transitions
    

In [7]:
html_files = os.listdir(articles_dir)
html_files = [html_file for html_file in html_files if re.search(r'html$',html_file) ]

html_files.remove('wordle-bot.html')
html_files.remove('index.html')
html_files.remove('year-in-interactive-graphics.html')
html_files.remove('year-in-interactive-storytelling.html')

print( f'The dataset contains {len(html_files)} articles.' )

The dataset contains 1011 articles.


In [12]:
annotations = []

article_id = 0 

for i,file in enumerate(html_files):
        
    article_id += 1
    
    article_title = re.sub( r'\.html' , '' , file )
    article_title = re.sub( r'[_-]' , ' ' , article_title )
    article_title = article_title.title()
    article_publisher='New York Times'

    
    #print(file)
    html_page = ''
    path = os.path.join(articles_dir,file)
    with open(path,encoding='utf-8') as fh:
        html_page = fh.read()

        
    if len(html_page)>0:
        try:

            # identify components
            components = []

            soup = BeautifulSoup( html_page ,"lxml")

            meta = soup.find_all('meta')


            for m in meta:
                m_content = m.get('content')
                m_property = m.get('property')
                
                if m_property == 'article:published_time':
                    year_publication = m_content[:4]



            body = soup.find('body' )

            for element in body.findChildren():

                if element.name == 'p':

                    attribute_value = get_attribute_value('class')

                    
                    if attribute_value == 'g-fg':
                        #print(f'Scroll text:\t{element.text.strip()}\n')
                        components.append( ('dynamic-text',element.text.strip())) 
                    
                    elif attribute_value == 'g-body':
                        #print(f'Body text:\n{element.text.strip()}')
                        components.append( ('text-flow',element.text.strip()))
                         

                    elif re.search( 'g-cue' , str(attribute_value) ):
                        components.append( ('subtitle',element.text.strip()))
                    elif re.search( 'svelte-1lru8e5' , str(attribute_value) ):
                        components.append( ('text-flow',element.text.strip()))
                        
                    elif re.search( 'introline' , str(attribute_value) ):
                        components.append( ('text-flow',element.text.strip()))
                    
                        
                    if re.search( r'^scroll\b' , element.text.strip() , re.IGNORECASE ):
                        components.append( ('scroll',element.text.strip()))

                    if re.search( r'^continue\s+reading\b' , element.text.strip() , re.IGNORECASE ):
                        components.append( ('scroll',element.text.strip()))

                if element.name == 'img':

                    attribute_value = get_attribute_value('class')

                    if re.search( r'g-overlay-play\s+', str(attribute_value) ):
                        components.append( ('dynamic-image-flow',element.text.strip()))
                        
#                     else:
#                         components.append( ('static-image',element.text.strip()))


                        
                if element.name == 'span':
                    
                    attribute_value = get_attribute_value('class')
                    
                    if re.search( r'audio-player-icon', str(attribute_value) ):
                        #print('Scroll \n')
                        components.append( ('button',element.text.strip()))


#                 if element.name == 'button':
                    
#                     attribute_value = get_attribute_value('class')
#                     print(attribute_value)
                    
#                     components.append( ('button',element.text.strip()))       
                        
                if element.name == 'div':

                    attribute_value = get_attribute_value('class')

                    if re.search( r'g-scroll-to-continue', str(attribute_value) ):
                        #print('Scroll \n')
                        components.append( ('scroll',element.text.strip()))

                    elif re.search( r'g-video\s+', str(attribute_value) ):
                        components.append( ('dynamic-image-flow',element.text.strip()))

                    elif re.search( r'g-audio', str(attribute_value) ):
                        text = element.text.strip()
                        text = re.sub( r'\s+' , ' ' , text )
                        if element.get('data-type') != 'audio':
                            components.append( ('audio',text))
                        
                    elif re.search( r'copy', str(attribute_value) ):
                        text = element.text.strip()
                        text = re.sub( r'\s+' , ' ' , text )
                        components.append( ('text-flow',text))
                        
                    elif re.search( r'\bg-graphic\b', str(attribute_value) ):
                        text = element.text.strip()
                        text = re.sub( r'\s+' , ' ' , text )
                        components.append( ('static-image' ,text))
                    
                    elif re.search( r'g-annotation', str(attribute_value) ) and not(re.search( r'g-annotation-inner', str(attribute_value) )):
                        text = element.text.strip()
                        text = re.sub( r'\s+' , ' ' , text )
                        components.append( ('dynamic-text',text))

                    elif re.search( r'text-unit', str(attribute_value) ):
                        components.append( ('text-flow',element.text.strip()))

                    elif re.search( r'g-image', str(attribute_value) ):
                        components.append( ('static-image',element.text.strip()))

                    elif re.search( r'g-scrollingparty-container', str(attribute_value) ):
                        components.append( ('dynamic-image-flow',element.text.strip()))

                    elif re.search( r'g-video', str(attribute_value) ):
                        components.append( ('dynamic-image-flow',element.text.strip()))
                        
                    elif re.search( r'g-asset', str(attribute_value) ):
                        contents = re.sub( r'\s+', '' , element.text.strip( ))
                        components.append( ('static-image',contents))
                        

                    elif re.search( r'g-edplaybtn', str(attribute_value) ):
                        components.append( ('button',element.text.strip()))
                        
                    elif re.search( r'tapbtn', str(attribute_value) ):
                        components.append( ('button',element.text.strip()))
                        
                    elif re.search( r'downarrow', str(attribute_value) ):
                        components.append( ('button',element.text.strip()))
                        

                        
            if len(components) == 0:

                scripts = soup.find_all('script')
                for s in scripts:
                    if re.search( 'var props =' , str(s) ):
                        lines = re.split( r'\n' , str(s) )
                        json_str = ''
                        for line in lines:
                            if re.search( 'var props =' , line):
                                json_str = re.sub( 'var props =' , '' , line ).strip()
                                index_slide = json_str.index('slides:') + len('slides:')
                                json_str = json_str[index_slide:]
                                json_str = re.sub(r'}$' , '' ,json_str.strip() ).strip()
                                json_str = re.sub( r'[\t\n]+' , '' , json_str )

                                index_slide = json_str.index('processedSlides:') 
                                json_str = json_str[:index_slide]
                                json_str = re.sub(',$', '' , json_str.strip() )


                        if len(json_str)>0:
                            #print(json_str)
                            out = open('article.json','w',encoding='utf-8')
                            out.write(json_str)
                            out.close()
                            json_data = json.loads(json_str)
                            for item in json_data:
                                if 'video' in item:
                                    components.append('video')

                                elif 'image' in item:
                                    #print(item['image'])
                                    components.append('static-image')

                                elif 'text' in item:
                                    #print(item['text'])
                                    components.append('text-flow')

            if len(components) > 0:
                 
                tr_in = 'none'
                tr_out = 'scroll'
                
                xml_file = re.sub( r'\.html' , '.xml' , file )
                out_path = os.path.join(annotation_dir,xml_file)
                out = open( out_path , 'w' , encoding = 'utf-8')

                out.write(f'<article title="{article_title}" publisher="{article_publisher}" year="{year_publication}" article_identifier="{(article_id):04d}">')
                current = ''
                for i,modality in enumerate(components):
                    if modality[0] == 'dynamic-image-flow':
                        screen_type = 'dynamic'
                    else:
                        screen_type = 'static'
                        
                    if modality[0] == 'scroll':                        
                        out.write( f'\n<scroll mode="{modality[0]}" identifier="{(i+1):04d}"/>')
                    
                    out.write( f'\n<screen identifier="{(i+1):04d}" mode="{modality[0]}" type="{screen_type}" transition_in="{tr_in}" transition_out="{tr_out}"/>')
                    component_contents = re.sub( r'--' , '' , str(modality[1]) )

                    out.write( f'\n<!--{component_contents} -->' )
                    tr_in = tr_out

                out.write('\n</article>')   
                
                out.close()
                                    
        except:
            print(f'Problem with {file}')
                                    

print('Done!')

Done!
